In [1]:
import numpy as np 
import pandas as pd 
import pyreadstat

In [2]:
# reading the states file
fp3 = "./metro cities/3_LASI_W1_Individual_metrocities_28-03-22.sav"
df_fp3, meta_fp3 = pyreadstat.read_sav(fp3, encoding="latin1")

In [3]:
# Function to get the variable name given a column label (case-insensitive)
def get_varname_by_label(meta, target_label):
    target_label = target_label.strip().lower()
    for var, label in meta.column_names_to_labels.items():
        if label is not None and label.strip().lower() == target_label:
            return var
    return None

# Define a function to recode yes/no responses (assuming 1=Yes, 2=No)
def recode_yes(x):
    return 1 if x == 1 else 0  # Convert 1 to 1, everything else to 0

# Defining Migration
---

| **Variable**     | **Type**      | **Coding / Description**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|------------------|---------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **migration_1**  | Categorical   | **Life‐course Migration Indicator** derived from comparing the respondent’s full place‐of‐birth location with their life‐course residential history. The full location is a concatenated string of country, state, district, and village/town.<br><br>• **0**: No migration – if the respondent’s childhood and adult residences match their place of birth or if the “years living continuously in this area” value equals 9993 (indicating “since birth”).<br>• **1**: Childhood migration – if any component of the childhood residence (up to age 14) differs from the place of birth.<br>• **2**: Adult migration – if no childhood migration is detected but any component of the adult residence (most of your adult life) differs from the place of birth. |
| **migration_2**  | Binary        | **Hard Threshold Indicator** based on the duration of residence in the current area.<br><br>• **1**: Migrant – if the respondent has lived in the current area for less than 20 years.<br>• **0**: Non‐migrant – if the respondent has lived in the current area for 20 years or more or if the “years living continuously” value equals 9993 (i.e. “since birth”).                                                     |
| **migration_3**  | Ordinal       | **Duration-Based Migration Indicator** using the number of years of continuous residence in the current area. The categories are defined as follows:<br><br>• **1**: 0–2 years<br>• **2**: 3–5 years<br>• **3**: 6–10 years<br>• **4**: 11–20 years<br>• **5**: 21–40 years<br>• **6**: More than 40 years<br>• **0**: Missing or undefined values                                                     |
| **migration_4**  | Categorical   | **Rural/Urban Migration Type** derived from comparing the rural/urban classification of the respondent’s place of birth with that of their last residence.<br><br>• **1**: Rural-to-Rural<br>• **2**: Rural-to-Urban<br>• **3**: Urban-to-Rural<br>• **4**: Urban-to-Urban<br>• **0**: Missing or undefined (if either classification is missing)                                               |

---



In [4]:
# params for definition 2
THRESHOLD = 20

# params for definition 3
BINS = [-np.inf, 2, 5, 10, 20, 40, np.inf]
LABELS = [1, 2, 3, 4, 5, 6]

# params for outcome - mental health (CIDI SF score method)
# Define the column names directly (based on the lowercase version of LASI dataset codes)
cols_cidi1 = ["mh204", "mh205", "mh206", "mh207", "mh208", "mh209", "mh210", "mh211"]
cols_cidi2 = ["mh217", "mh218", "mh219", "mh220", "mh221", "mh222"]

# Screening questions
col_MH201 = "mh201"  # Screening for dysphoria
col_MH214 = "mh214"  # Screening for anhedonia

CIDI_1_THRESHOLD = 3 # Depression: if CIDI_1 score >=3, then 1 else 0

In [5]:
# Get variable name for "Since how many years living continuously in this area"
col_name_years = get_varname_by_label(meta_fp3, "Since how many years living continuously in this area")
if col_name_years is None:
    raise ValueError("Column label 'Since how many years living continuously in this area' not found in fp3.sav metadata.")

# Convert the column to numeric (handling errors)
years_living = pd.to_numeric(df_fp3[col_name_years], errors='coerce')

# Create a new DataFrame for migration_2 and migration_3
df_input = pd.DataFrame()


In [6]:

# Retrieve variable names for place of birth
col_birth_country = get_varname_by_label(meta_fp3, "Place of birth-country")
col_birth_state   = get_varname_by_label(meta_fp3, "Place of birth-state")
col_birth_district = get_varname_by_label(meta_fp3, "Place of birth-district")
col_birth_village  = get_varname_by_label(meta_fp3, "Place of birth-village/town")

# Retrieve variable names for childhood residence
col_child_country = get_varname_by_label(meta_fp3, "Lived most of your childhood-Country")
col_child_district = get_varname_by_label(meta_fp3, "Lived most of your childhood-distirct")
col_child_state   = get_varname_by_label(meta_fp3, "Lived most of your childhood-state")
col_child_village  = get_varname_by_label(meta_fp3, "Lived most of your childhood-village/town")

# Retrieve variable names for adult residence
col_adult_country = get_varname_by_label(meta_fp3, "Lived most of your adult life-country")
col_adult_state   = get_varname_by_label(meta_fp3, "Lived most of your adult life-state")
col_adult_district = get_varname_by_label(meta_fp3, "Lived most of your adult life-district")
col_adult_village  = get_varname_by_label(meta_fp3, "Lived most of your adult life-village/town")

# Check that all required variables were found
for name, col in [("Place of birth-country", col_birth_country),
                  ("Place of birth-state", col_birth_state),
                  ("Place of birth-district", col_birth_district),
                  ("Place of birth-village/town", col_birth_village),
                  ("Lived most of your childhood-Country", col_child_country),
                  ("Lived most of your childhood-distirct", col_child_district),
                  ("Lived most of your childhood-state", col_child_state),
                  ("Lived most of your childhood-village/town", col_child_village),
                  ("Lived most of your adult life-country", col_adult_country),
                  ("Lived most of your adult life-state", col_adult_state),
                  ("Lived most of your adult life-district", col_adult_district),
                  ("Lived most of your adult life-village/town", col_adult_village)]:
    if col is None:
        raise ValueError(f"Column label '{name}' not found in fp3.sav metadata.")

# Extract the relevant columns and clean the data
def clean_series(series):
    return series.astype(str).str.strip().str.lower().fillna("")

birth_country = clean_series(df_fp3[col_birth_country])
birth_state = clean_series(df_fp3[col_birth_state])
birth_district = clean_series(df_fp3[col_birth_district])
birth_village = clean_series(df_fp3[col_birth_village])

child_country = clean_series(df_fp3[col_child_country])
child_state = clean_series(df_fp3[col_child_state])
child_district = clean_series(df_fp3[col_child_district])
child_village = clean_series(df_fp3[col_child_village])

adult_country = clean_series(df_fp3[col_adult_country])
adult_state = clean_series(df_fp3[col_adult_state])
adult_district = clean_series(df_fp3[col_adult_district])
adult_village = clean_series(df_fp3[col_adult_village])

# Create concatenated location strings (using a separator to clearly mark boundaries)
birth_location = birth_country + "|" + birth_state + "|" + birth_district + "|" + birth_village
child_location = child_country + "|" + child_state + "|" + child_district + "|" + child_village
adult_location = adult_country + "|" + adult_state + "|" + adult_district + "|" + adult_village

# Compute migration indicators:
# Childhood Migration Indicator: 1 if any level differs between place of birth and childhood residence, else 0.
child_migration = (birth_location != child_location).astype(int)

# Adult Migration Indicator: 1 if any level differs between place of birth and adult residence, else 0.
adult_migration = (birth_location != adult_location).astype(int)

# Create a new DataFrame with the concatenated locations and migration indicators
df_life_course_migration = pd.DataFrame({
    "Birth Location": birth_location,
    "Childhood Location": child_location,
    "Childhood Migration Indicator": child_migration,
    "Adult Location": adult_location,
    "Adult Migration Indicator": adult_migration
})

# df_input["migration_1"] = np.where(
#     years_living == 9993,
#     "0",
#     np.where(child_migration == 1, "child_migration",
#              np.where(adult_migration == 1, "adult_migration", "0"))
# )

df_input["migration_1"] = np.where(
    years_living == 9993,
    "0",
    np.where(child_migration == 1, 1,
             np.where(adult_migration == 1, 2, 0))
)



# Create migration_2: 0 if years >= THRESHOLD, else 1
df_input["migration_2"] = np.where(years_living >= THRESHOLD, 0, 1)

# Use pd.cut, and fill NaN values with a default category (e.g., 0 for unknown)
df_input["migration_3"] = pd.cut(years_living, bins=BINS, labels=LABELS, right=True)

# Convert migration_3 to integer, replacing NaN with 0 (or any default category you prefer)
df_input["migration_3"] = df_input["migration_3"].cat.add_categories(0).fillna(0).astype(int)



In [7]:
# Get variable names by column labels (using your previously defined function)
col_DM017 = get_varname_by_label(meta_fp3, "Place of residence")
col_DM018 = get_varname_by_label(meta_fp3, "Place of last residence-village/town")

# Extract the columns for DM017 and DM018
# (These variables are assumed to contain numeric codes: 1 for village, 2 for town)
place_birth = df_fp3[col_DM017]
place_last_res = df_fp3[col_DM018]

# For clarity, create a new DataFrame with only the two variables:
df_migration_type = pd.DataFrame({
    "Place of Birth": place_birth,
    "Place of Last Residence": place_last_res
})

# Define a function to classify migration type based on rural/urban codes
# def classify_migration(row):
#     pb = row["Place of Birth"]
#     plr = row["Place of Last Residence"]
#     # Check for missing values (if not coded as 1 or 2, you might want to handle them separately)
#     if pd.isna(pb) or pd.isna(plr):
#         return "0"
#     if pb == 1 and plr == 1:
#         return "Rural-to-Rural"
#     elif pb == 1 and plr == 2:
#         return "Rural-to-Urban"
#     elif pb == 2 and plr == 1:
#         return "Urban-to-Rural"
#     elif pb == 2 and plr == 2:
#         return "Urban-to-Urban"
#     else:
#         return np.nan

def classify_migration(row):
    pb = row["Place of Birth"]
    plr = row["Place of Last Residence"]
    # Check for missing values (if not coded as 1 or 2, you might want to handle them separately)
    if pd.isna(pb) or pd.isna(plr):
        return 0
    if pb == 1 and plr == 1:
        return 1
    elif pb == 1 and plr == 2:
        return 2
    elif pb == 2 and plr == 1:
        return 3
    elif pb == 2 and plr == 2:
        return 4
    else:
        return np.nan

# Apply the function to each row to create a new column 'Migration Type'
df_input["migration_4"] = df_migration_type.apply(classify_migration, axis=1)

In [8]:
# Compute CIDI_1 (dysphoria score)
df_fp3["MH201_binary"] = df_fp3[col_MH201].apply(recode_yes)
df_input["CIDI_1"] = df_fp3[cols_cidi1].applymap(recode_yes).sum(axis=1)
df_input.loc[df_fp3["MH201_binary"] == 0, "CIDI_1"] = 0  # Set to 0 if screening was No

# Compute CIDI_2 (anhedonia score)
df_fp3["MH214_binary"] = df_fp3[col_MH214].apply(recode_yes)
df_input["CIDI_2"] = df_fp3[cols_cidi2].applymap(recode_yes).sum(axis=1)
df_input.loc[df_fp3["MH214_binary"] == 0, "CIDI_2"] = 0  # Set to 0 if screening was No

# Create binary depression variable (1 if CIDI_1 >= CIDI_1_THRESHOLD, else 0)
df_input["depression"] = df_input["CIDI_1"].apply(lambda x: 1 if x >= CIDI_1_THRESHOLD else 0)


/tmp/ipykernel_18816/821456790.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input["CIDI_1"] = df_fp3[cols_cidi1].applymap(recode_yes).sum(axis=1)
/tmp/ipykernel_18816/821456790.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input["CIDI_2"] = df_fp3[cols_cidi2].applymap(recode_yes).sum(axis=1)


In [9]:
print(df_input.head())

  migration_1  migration_2  migration_3  migration_4  CIDI_1  CIDI_2  \
0           2            1            3            4       0       0   
1           2            1            4            3       0       0   
2           2            1            4            3       0       0   
3           2            0            5            3       0       0   
4           2            0            5            3       0       0   

   depression  
0           0  
1           0  
2           0  
3           0  
4           0  


# Controlling Variables

In [10]:


# --- 1. Education ---
# Column codes (in lowercase) are used directly.
# dm006: Ever attended school
# dm008: Highest level of education
# Rule:
#   - If dm006 != 1, education = 0.
#   - Otherwise, if dm008 is 1,2,3,or 4, education = that value.
#     If dm008 is 5,6,7,8, or 9, education = 5.
def compute_education(row):
    try:
        attended = int(row["dm006"])
    except:
        attended = 0
    if attended != 1:
        return 0
    try:
        edu = int(row["dm008"])
    except:
        return 0
    if edu in [1, 2, 3, 4]:
        return edu
    elif edu in [5, 6, 7, 8, 9]:
        return 5
    else:
        return 0

df_input["education_level"] = df_fp3.apply(compute_education, axis=1).fillna(0)

# --- 2. Living Arrangements ---
# fs329: Are you satisfied with your current living arrangements?
# Original coding: 1 = Strongly satisfied, 2 = Satisfied, 3 = Neither, 4 = Dissatisfied, 5 = Strongly dissatisfied.
# We want: 1 = Strongly dissatisfied, 5 = Strongly satisfied.
# Compute new value as: new_value = 6 - original_value.
def compute_living(val):
    try:
        v = int(val)
    except:
        return 0
    if v in [1, 2, 3, 4, 5]:
        return 6 - v
    else:
        return 0

df_input["living_arrangements"] = df_fp3["fs329"].apply(compute_living).fillna(0)

# --- 3. MPCE Quintile ---
# We assume the column name is exactly "mpce quintile" (in lowercase)
col_mpce = get_varname_by_label(meta_fp3, "MPCE quintile")
df_input["mpce_quintile"] = df_fp3[col_mpce].fillna(0).astype(int)

# --- 4. Age Category ---
# dm005: Age at last birthday (in complete years)
# Create categories:
#   45-55 -> 1, 55-65 -> 2, 65-75 -> 3, 75-85 -> 4, >85 -> 5.
df_fp3["age_numeric"] = pd.to_numeric(df_fp3["dm005"], errors="coerce").fillna(0)
bins = [45, 55, 65, 75, 85, np.inf]
labels = [1, 2, 3, 4, 5]
df_fp3["age_category"] = pd.cut(df_fp3["age_numeric"], bins=bins, labels=labels, right=False)
# Replace missing age category with 0 and convert to integer
df_input["age_category"] = df_fp3["age_category"].cat.add_categories(0).fillna(0).astype(int)

# --- 5. Currently Working ---
# we004: Are you currently working? (Assumed coding: 1 = Yes, otherwise No)
# df_fp3["currently_working"] = df_fp3["we004"].apply(lambda x: 1 if str(x).strip() == "1" else 0).fillna(-1)

# --- Create New DataFrame ---
# Select only the new columns we need.
# df_new = df_[["education", "living_arrangements", "mpce_quintile", "age_category"]]

# Display the first few rows of the new DataFrame
print(df_input.head())


  migration_1  migration_2  migration_3  migration_4  CIDI_1  CIDI_2  \
0           2            1            3            4       0       0   
1           2            1            4            3       0       0   
2           2            1            4            3       0       0   
3           2            0            5            3       0       0   
4           2            0            5            3       0       0   

   depression  education_level  living_arrangements  mpce_quintile  \
0           0                2                    4              3   
1           0                0                    4              1   
2           0                0                    4              1   
3           0                4                    4              3   
4           0                4                    4              3   

   age_category  
0             2  
1             2  
2             1  
3             2  
4             2  


In [11]:
counts = df_input["migration_1"].value_counts()
print(counts)
print()
counts = df_input["migration_2"].value_counts()
print(counts)
print()
counts = df_input["migration_3"].value_counts()
print(counts)
print()
counts = df_input["migration_4"].value_counts()
print(counts)
print()
counts = df_input["depression"].value_counts()
print(counts)
print()
counts = df_input["education_level"].value_counts()
print(counts)
print()
counts = df_input["living_arrangements"].value_counts()
print(counts)
print()
counts = df_input["mpce_quintile"].value_counts()
print(counts)
print()
counts = df_input["age_category"].value_counts()
print(counts)
print()


migration_1
0    3016
2    1235
1     222
Name: count, dtype: int64

migration_2
0    3792
1     681
Name: count, dtype: int64

migration_3
6    2182
5    1394
4     542
3     211
2      87
1      51
0       6
Name: count, dtype: int64

migration_4
3    1618
0    1556
4    1299
Name: count, dtype: int64

depression
0    4321
1     152
Name: count, dtype: int64

education_level
0    1315
5     944
2     647
4     644
3     631
1     292
Name: count, dtype: int64

living_arrangements
4    2549
5    1007
3     743
2     104
0      50
1      20
Name: count, dtype: int64

mpce_quintile
4    962
2    951
3    941
5    857
1    762
Name: count, dtype: int64

age_category
1    1630
2    1150
3     848
0     467
4     297
5      81
Name: count, dtype: int64



In [ ]:
df_input.to_csv('df_input_metro.csv', index=False)


# Documentation for Geetika, Siddharth, Vaishnavi

| **Variable**           | **Type**        | **Code / Description**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | **Dependent/Independent** |
|------------------------|-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------|
| **migration_1**        | Categorical     | **Life‐course migration indicator** <br>• **0**: No migration <br>• **1**: Childhood migration <br>• **2**: Adult migration                                                                                                                                                                                                                                                                                                                                                         | Independent                |
| **migration_2**        | Binary          | **Hard threshold indicator based on duration of residence:**<br>• **1**: Migrant (respondent has lived continuously in the current area for less than 20 (THRESHOLD) years).<br>• **0**: Non‐migrant (respondent has lived 20 (THRESHOLD) years or more in the current area).                                                                                                                                                                                                                                              | Independent                |
| **migration_3**        | Ordinal         | **Duration‐based migration indicator** (using “Since how many years living continuously in this area”):<br>• **1**: 0–2 years<br>• **2**: 3–5 years<br>• **3**: 6–10 years<br>• **4**: 11–20 years<br>• **5**: 21–40 years<br>• **6**: More than 40 years                                                                                                                                                                                                                                                                    | Independent                |
| **migration_4**        | Categorical     | **Rural/Urban migration type** (based on rural/urban codes for “Place of Birth” and “Place of Last Residence”):<br>• **1**: Rural-to-Rural<br>• **2**: Rural-to-Urban<br>• **3**: Urban-to-Rural<br>• **4**: Urban-to-Urban<br>• **0**: Missing/undefined (if either location is missing)                                                                                                                                                                                                                                  | Independent                |
| **CIDI_1**             | Numeric (Score) | **Depression score from CIDI-SF (Part 1):** Sum of “Yes” responses to items MH204–MH211.                                                                                                                                                                                                                                                                                                                                                                                           | Ignore this                |
| **CIDI_2**             | Numeric (Score) | **Depression score from CIDI-SF (Part 2):** Sum of “Yes” responses to items MH217–MH222. This score is used as a complementary measure of depression.                                                                                                                                                                                                                                                                                                                             | Ignore this                |
| **Depression**         | Binary          | **Depression outcome variable:**<br>• **1**: The respondent is considered at risk for depression (CIDI_1 score ≥ 3).<br>• **0**: Otherwise.                                                                                                                                                                                                                                                                                                                                         | Dependent                |
| **education**          | Ordinal         | **Educational attainment** (derived from DM008, with DM006 “Ever attended school” check):<br>• **0**: Did not attend school or DM006 ≠ 1<br>• **1**: Less than primary school (Standard 1–4)<br>• **2**: Primary school completed (Standard 5–7)<br>• **3**: Middle school completed (Standard 8–9)<br>• **4**: Secondary/Matriculation completed<br>• **5**: Higher secondary/Intermediate and above (Diploma, Graduate, Postgraduate, or Professional course)                | Independent and Controlling Variable                |
| **living_arrangements**| Ordinal         | **Satisfaction with current living arrangements** (from FS329):<br>• **1**: Strongly dissatisfied<br>• **2**: Dissatisfied<br>• **3**: Neither satisfied nor dissatisfied<br>• **4**: Satisfied<br>• **5**: Strongly satisfied                                                                                                                                                                                                                                                        | Independent and Controlling Variable                |
| **mpce_quintile**      | Ordinal         | **Monthly Per Capita Consumption Expenditure (MPCE) Quintile**:<br>• **1**: Lowest quintile<br>• **2**: Second quintile<br>• **3**: Middle quintile<br>• **4**: Fourth quintile<br>• **5**: Highest quintile                                                                                                                                                                                                                                                                         | Independent and Controlling Variable                |
| **age_category**       | Ordinal         | **Age group** (derived from “Age at last birthday”):<br>• **1**: 45–55 years<br>• **2**: 55–65 years<br>• **3**: 65–75 years<br>• **4**: 75–85 years<br>• **5**: Over 85 years                                                                                                                                                                                                                                                                                                       | Independent and Controlling Variable                |

---


# Full Documentation

| **Variable**           | **Type**        | **Code / Description**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | **Dependent/Independent** |
|------------------------|-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------|
| **migration_1**        | Categorical     | **Life‐course migration indicator** (derived using “Since how many years living continuously in this area” plus life‐course residence data):<br>• **0**: No migration – either the respondent has lived in the same location throughout life or the “years living” answer is coded as 9993 (“since birth”).<br>• **1**: Childhood migration – if any component (country, state, district, or village/town) of the childhood residence differs from the corresponding component of the place of birth.<br>• **2**: Adult migration – if no childhood migration is detected but any component of the adult residence differs from the corresponding component of the place of birth. | Independent |
| **migration_2**        | Binary          | **Hard threshold indicator based on duration of residence:**<br>• **1**: Migrant (respondent has lived continuously in the current area for less than 20 years).<br>• **0**: Non‐migrant (respondent has lived 20 years or more in the current area).                                                                                                                                                                                                                                                        | Independent |
| **migration_3**        | Ordinal         | **Duration‐based migration indicator** (using “Since how many years living continuously in this area”):<br>• **1**: 0–2 years<br>• **2**: 3–5 years<br>• **3**: 6–10 years<br>• **4**: 11–20 years<br>• **5**: 21–40 years<br>• **6**: More than 40 years                                                                                                                                                                                                                                  | Independent |
| **migration_4**        | Categorical     | **Rural/Urban migration type** (based on rural/urban codes for “Place of Birth” and “Place of Last Residence”):<br>• **1**: Rural-to-Rural<br>• **2**: Rural-to-Urban<br>• **3**: Urban-to-Rural<br>• **4**: Urban-to-Urban<br>• **0**: Missing/undefined (if either location is missing)                                                                                                                                                                                                                           | Independent |
| **CIDI_1**             | Numeric (Score) | **Depression score from CIDI-SF (Part 1):** Sum of “Yes” responses to items MH204–MH211. A score of 3 or higher indicates risk of depression.                                                                                                                                                                                                                                                                                                                                       | Ignore this |
| **CIDI_2**             | Numeric (Score) | **Depression score from CIDI-SF (Part 2):** Sum of “Yes” responses to items MH217–MH222. This score is used as a complementary measure of depression.                                                                                                                                                                                                                                                                                                                               | Ignore this |
| **Depression**         | Binary          | **Depression outcome variable:**<br>• **1**: The respondent is considered at risk for depression (CIDI-SF score ≥ 3).<br>• **0**: Otherwise.                                                                                                                                                                                                                                                                                                                                        | Dependent |
| **education**          | Ordinal         | **Educational attainment** (derived from DM008, with DM006 “Ever attended school” check):<br>• **0**: Did not attend school or DM006 ≠ 1<br>• **1**: Less than primary school (Standard 1–4)<br>• **2**: Primary school completed (Standard 5–7)<br>• **3**: Middle school completed (Standard 8–9)<br>• **4**: Secondary/Matriculation completed<br>• **5**: Higher secondary/Intermediate and above (Diploma, Graduate, Postgraduate, or Professional course)                            | Independent and Controlling Variable |
| **living_arrangements**| Ordinal         | **Satisfaction with current living arrangements** (from FS329):<br>• **1**: Strongly dissatisfied<br>• **2**: Dissatisfied<br>• **3**: Neither satisfied nor dissatisfied<br>• **4**: Satisfied<br>• **5**: Strongly satisfied                                                                                                                                                                                                                                                        | Independent and Controlling Variable |
| **mpce_quintile**      | Ordinal         | **Monthly Per Capita Consumption Expenditure (MPCE) Quintile**:<br>• **1**: Lowest quintile<br>• **2**: Second quintile<br>• **3**: Middle quintile<br>• **4**: Fourth quintile<br>• **5**: Highest quintile                                                                                                                                                                                                                                                                         | Independent and Controlling Variable |
| **age_category**       | Ordinal         | **Age group** (derived from “Age at last birthday”):<br>• **1**: 45–55 years<br>• **2**: 55–65 years<br>• **3**: 65–75 years<br>• **4**: 75–85 years<br>• **5**: Over 85 years                                                                                                                                                                                                                                                                                                         | Independent and Controlling Variable |


